In [1]:
import tensorflow as tf

In [2]:
matrix_1 = tf.constant([[3,3]])
matrix_2 = tf.constant([[2],
                       [2]])
product = tf.matmul(matrix_1, matrix_2)  # == np.dot(m_1,m_2)   矩阵相乘

In [3]:
#method 1
sess = tf.Session() #with tf.Session() as sess
result = sess.run(product)
print(result)
sess.close()   

[[12]]


In [4]:
state = tf.Variable(0, name='counter')  # tf定义变量
state.name

'counter:0'

In [5]:
one_tf = tf.constant(1) #tf定义常量
new_val = tf.add(state,one_tf)
update = tf.assign(state, new_val)  # state = new_val

In [6]:
init = tf.global_variables_initializer()  #定义则需要初始化！！  且必须要用session.run(init)

In [7]:
with tf.Session() as sess:
    sess.run(init)
    for _ in range(2):
        sess.run(update)
        print(sess.run(state))

1
2


In [8]:
input_1 = tf.placeholder(tf.float32, name='x_input')
input_2 = tf.placeholder(tf.float32, name='y_input')
output = tf.multiply(input_1, input_2)  # 元素相乘

In [9]:
with tf.Session() as sess:
    print(sess.run(output, feed_dict={input_1:[7.0], input_2:[11.0]}))

[77.]


In [10]:
def add_layer(inputs, in_size, out_size, activation_func=None):
    layer_name = "layer_eric"
    with tf.name_scope('layer'):
        with tf.name_scope('Weights'):
            Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='weight')
            tf.summary.histogram(layer_name+'/weights', Weights)
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1,out_size]) + 0.1, name='biases')
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
        if activation_func is None:
            output = Wx_plus_b
        else:
            output = activation_func(Wx_plus_b,)
        return output

In [11]:
import numpy as np

In [12]:
x_data = np.linspace(-1,1,300)[:,np.newaxis].astype(np.float32)  # value in (-1,1), 300 values
noise = np.random.normal(0,0.05, x_data.shape)
y_data = np.square(x_data) - 0.5 + noise

In [13]:
with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32,[None, 1], name='x_input')
    ys = tf.placeholder(tf.float32,[None, 1], name='y_input')

In [14]:
l1 = add_layer(xs, 1, 10, activation_func=tf.nn.relu)
prediction = add_layer(l1, 10, 1, activation_func=None)

In [15]:
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction), reduction_indices=[1])) #求平均值， 求和， 求方差，  reduction_indices=[1] 结果压缩方向->按行求和
tf.summary.scalar('loss', loss)

<tf.Tensor 'loss_1:0' shape=() dtype=string>

In [16]:
with tf.name_scope('train_step'):
    train_step = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)

In [17]:
init = tf.global_variables_initializer()

In [18]:
import matplotlib.pyplot as plt

In [19]:
with tf.Session() as sess:
    sess.run(init)
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter("logs/", tf.get_default_graph())
    for i in range(1000):
        sess.run(train_step, feed_dict={xs:x_data, ys:y_data})
        if i%50 == 0:
            #print(sess.run(loss,feed_dict={xs:x_data, ys:y_data}))
            #prediction_value = sess.run(prediction, feed_dict={xs:x_data})
            #lines = ax.plot(x_data, prediction_value, 'red')
            #ax.lines.remove(lines[0])
            results = sess.run(merged, feed_dict={xs:x_data, ys:y_data})
            writer.add_summary(results, i)  #i means step